[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aprendizaje-automatico-dc-uba-ar/material/blob/main/notebooks/notebook_11_clustering-published.ipynb) (Este botón no anda, tenemos que ver dónde colgaremos las cosas)

In [10]:
from IPython.display import Latex, display
import numpy as np

# Parte 1: Evaluación de modelos.

Este notebook contiene los ejercicios de la primera parte del taller `Evaluación de modelos, series temporales y toma de decisiones` desarrollado para [Escuela Bayes Plurinacional](https://bayesplurinacional.org/) SALTA 2024.

**Ejercicios**:
- 1.1 Modelo Base vs Modelo Monty Hall
    - 1.1.1 Definir sus distribuciones condicionales *a priori*
    - 1.1.2 Simular datos con el modelo Monty Hall
    - 1.1.3 Calcular la secuencia de predicción que hacen los modelo de los datos
    - 1.1.4 Expresar intuitivamente la diferencia de desempeño predictivo de los modelos
    - 1.1.5 Calcular cómo se va actualizando la creencia de los modelos a medida que vamos agregando datos
    - 1.1.6 Graficar el valor del posterior a medida que se observan nuevos episodios
    - 1.1.7 ¿Que ocurriría con la evaluación de modelos si en los datos hubiera al menos una pista sobre la caja cerrada? 
- 1.2. Modelo AcausaB vs BcausaA
- 1.3 


## Ejemplo 1.1: Modelo "Base" ($M_0$) vs Modelo "Monty Hall" ($M_1$)
En la siguiente figura se puede observar la especificación gráfica del modelo "Monty Hall" (izquierda) y el modelo "Base" (derecha) visto en la presentación. Abajo de ellos se muestra la distribución de creencias *a posteriori* sobre la posición del regalo luego de que hayamos elegido cerrar la caja $1$ y nos mostraran que la caja $2$ no estaba el regalo, $P(r|s=2,c=1)$

<center>
<img src="https://raw.githubusercontent.com/BayesDeLasProvinciasUnidasDelSur/static/master/curso/2024/MontyHall.png" alt="drawing" width="50%"/>
</center>

El modelo Monty Hall supone que la pista $s$ no puede señalar la caja en la que se encuentra el regalo $s\neq r$ ni la caja que hemos cerrado previamente $s\neq c$. En cambio, el modelo Base supone que la pista $s$ no puede señalar únicamente a la caja en la que se encuentra el regalo $s\neq r$, pudiendo señalar cualquiera de las otras dos cajas, sin tener en cuenta si hemos cerrado una de ellas previamente.

Quisiéramos actualizar nuestras creencias sobre los modelos causales alternativos luego de observar los datos, $P(\text{Modelo}|\text{Datos})$.

$$P(\text{Modelo}|\text{Datos}) = \frac{P(\text{Datos}|\text{Modelo})P(\text{Modelo})}{P(\text{Datos})}$$

Debemos calcular:
- La predicción que hace el modelo sobre los datos: $P(\text{Datos}|\text{Modelo})$
- La predicción de los datos realizada con la contribución de todos los modelos: $P(\text{Datos})$
- La creencia previa "honesta" sobre los modelos: $P(\text{Modelo})$

Si antes de ver los datos no tenemos preferencia por ninguno de los modelos podemos dividir nuestra creencia en partes iguales.

$$P(\text{Modelo}) = 0.5$$

Recordemos que por la regla de la suma (princpio de integridad), la predicción de los datos se realiza con la contribución de todas las hipótesis.

$$P(\text{Datos}) \overset{\genfrac{}{}{0pt}{}{\text{Regla de}}{\text{la suma}} }{=} \sum_{\text{Modelo}} P(\text{Modelo},\text{Datos}) \overset{\genfrac{}{}{0pt}{}{\text{Regla de}}{\text{la producto}} }{=} \sum_{\text{Modelo}} P(\text{Datos}|\text{Modelo}) P(\text{Modelo})$$

Luego, lo único que nos hace falta calcular es la predicción que hace cada uno de los modelos con la contribución de todas sus hipótesis internas.

$$P(\text{Datos} = \{\underbrace{c_1, s_1, r_1}_{\genfrac{}{}{0pt}{}{\text{Primer}}{\text{episodio}}}, \underbrace{c_2, s_2, r_2}_{\genfrac{}{}{0pt}{}{\text{Segundo}}{\text{episodio}}},  \,  \dots \, \underbrace{c_T, s_T, r_T}_{\genfrac{}{}{0pt}{}{\text{T-ésimo}}{\text{episodio}}}  \}|\text{Modelo})$$

Un episodio es la secuencias completa de observaciones de todas las variables ocultas de un modelo. Al iniciar un episoidio, todas las variables son ocultas (hipótesis). Al final el episodio, todas son observables. En este ejemplo, vamos a considerar que al interior de un episodio $t$ (con $t \in \{1 \dots T\}$) las observaciones llegan siempre en el mismo orden: observamos primero la caja que se cierra $c_t$, luego la pista $s_t$ y finalmente la posición del regalo $r_t$. Luego, la predicción que hace un modelo sobre esos datos puede escribirse sel siguiente modo,

$$P(\text{Datos}=\{c_t,s_t,r_t \}|Modelo) = P(c_t|Modelo)P(s_t|c_t,Modelo)P(r_t|s_t,c_t,Modelo) $$

Como ninguno de los dos modelos vincula los episodios entre sí (ninguno usa los datos de un episodio para predecir lo datos en otro episodio), podemos descomponer la predicción sobre el conjunto de datos sobre todos los episodios como el producto de las predicciones que lo modelos hacen al interior de cada episodio.

$$P(\text{Datos} |\text{Modelo}) = \prod_{t\in \{1,\dots,T \}} P(\underbrace{c_t, s_t, r_t}_{\genfrac{}{}{0pt}{}{\text{t-ésimo}}{\text{episodio}}}|\text{Modelo}) $$

Tenemos todo para calcular la probabilidad de los modelos dado los datos. Defimamos primero las distribución condiciones *a priori* honesta de cada uno los modelos.   


### **Ejercicio 1.1.1**: Definir sus distribuciones condicionales *a priori*

La espcificación gráfica del modelo Base $M_0$ define la distribución conjunta *a priori* sobre las tres hipótesis, $P(r,c,s|M_0)$, como una descomposición entre las siguientes tres probabilidades condicionales.

$$P(r,c,s|M_0) = P(r|M_0)P(c|M_0)P(s|r,M_0) $$

La espcificación gráfica del modelo Monty Hall $M_1$ define la distribución conjunta *a priori* sobre las tres hipótesis, $P(r,c,s|M_0)$, como una descomposición entre las siguientes tres probabilidades condicionales.

$$P(r,c,s|M_1) = P(r|M_0)P(c|M_0)P(s|r,c,M_1) $$

Las distribuciones condicionales *a priori* sobre $r$ y $c$ son iguales en ambos modelos.

$$
P(r|M) = \begin{array}{c|c|c}
  r=1 & r=2 & r=3 \\
  \hline
  1/3 & 1/3 & 1/3 \\
\end{array}

\hspace{2cm}

P(c|M) = \begin{array}{c|c|c}
  c=1 & c=2 & c=3 \\
  \hline
  1/3 & 1/3 & 1/3 \\
\end{array}
$$

La única diferencia entre los modelos aparece en la distribución condicional sobre la pista. En el modelo Base solo depende del regalo $r$.

$$
P(s|r,M_0) = \begin{array}{c|c|c|c}
  & s=1 & s=2 & s=3 \\ \hline
 r=1 & 0 & 1/2 & 1/2 \\ \hline
 r=2 & 1/2 & 0 & 1/2 \\ \hline 
 r=3 & 1/2 & 1/2 & 0 \\ \hline
\end{array}
$$

Y en el modelo Monty Hall depende del regalo $r$ y la caja cerrada $c$. Para simplificar, mostraremos los valores cuando $c=1$.

$$
P(s|r,c=1,M_0) = \begin{array}{c|c|c|c}
  & s=1 & s=2 & s=3 \\ \hline
 r=1 & 0 & 1/2 & 1/2 \\ \hline
 r=2 & 0 & 0 & 1 \\ \hline 
 r=3 & 0 & 1 & 0 \\ \hline
\end{array}
$$

Notar que cada renglón suma 1, pues cada condicional representa una distribución de probabilidad distinta.

In [8]:
# Probabilidad del regalo
pr = [1/3, 1/3, 1/3] # P(r) = pr[r]

# Probabilidad de cerrar
pc = [1/3, 1/3, 1/3] # P(c) = pc[c]
# Probabilidad de la pista modelo Base M0

# P(s|r,M0) = ps_rM0[r][s] 
ps_rM0=[[  0, 1/2, 1/2],   # r=1
        [1/2,   0, 1/2],   # r=2
        [1/2, 1/2,   0]]   # r=3
# Probabilidad de la pista modelo Monty Hall M1

# P(s|r,c,M1) = ps_rcM1[c][r][s] 
ps_rcM1=[[[  0, 1/2, 1/2],  # r=1, c=1
          [  0,   0,   1],  # r=2, c=1
          [  0,   1,   0]], # r=3, c=1
         [[  0,   0,   1],  # r=1, c=2
          [1/2,   0, 1/2],  # r=2, c=2
          [  1,   0,   0]], # r=3, c=2
         [[  0,   1,   0],  # r=1, c=3
          [  1,   0,   0],  # r=2, c=3
          [1/2, 1/2,   0]]] # r=3, c=3

# Probabilidad del modelo
# P(m) = pM[m]
pM = [1/2, 1/2]


### **Ejercicio 1.1.2**: Simular datos con el modelo Monty Hall

Antes de evaluar los modelos necesitamos un conjunto de datos que provengan de la realidad subyacente oculta. Podríamos buscar los datos reales del programa de televisión Monty Hall y revisar si efectivamente el modelo Monty Hall propuesto es mejor que el modelo Base. Para simplificar vamos a suponer que nuestro modelo Monty Hall representa perfectamente la realidad causal subyacente y a generar los datos usando de nuestro propio modelo Monty Hall. Luego, vamos a usar ese conjunto de datos para evaluar los dos modelos. Es de esperar que el modelo que generó los datos tenga mejor desempeño predictivo que el modelo alternativo (en el ejercicio 1.2 veremos que esto no siempre esto es así). Pero también nos interesa ver cuántos datos necesitamos para tener relativa certeza de que la realidad causal subyacente se correponde con el modelo Monty Hall. 

In [11]:
# SOLUCION 1.2 

np.random.seed(0)

T = 16 # Cantidad total de episodios
Datos = [] # La lista de los datos
for t in range(T): # Itero por episodio
    # Genero los datos 
    r = np.random.choice(3, p=pr)
    c = np.random.choice(3, p=pc)
    s = np.random.choice(3, p=ps_rcM1[c][r])
    Datos.append((c,s,r)) # Agrega el episodio como una tupla (c,s,r)

print(Datos)


[(2, 0, 1), (1, 2, 1), (2, 0, 1), (2, 0, 1), (2, 0, 1), (0, 2, 0), (2, 1, 2), (1, 0, 2), (1, 2, 0), (1, 0, 2), (2, 1, 0), (0, 2, 1), (1, 2, 1), (1, 0, 2), (0, 1, 2), (0, 1, 2)]


### **Ejercicio 1.1.3**: Calcular la secuencia de predicción que hacen los modelo de los datos 

Guardar en las listas `pDatos_M0` y `pDatos_M1` las secuencias de predicciones que hacen cada uno de los modelos sobre la secuencia de episodios. Inicializamos ambas listas con un $1$ (predicción perfecta) en para representar el estado previo a comenzar a predecir los episodios 

In [12]:
# OCULTAR

# Inicializacion de la secuencia de predicciones de los modelos
pDatos_M0 = [1] # Del modelo 0: No Monty Hall
pDatos_M1 = [1] # Del modelo 1: Monty Hall
for t in range(T): # Itero sobre episodios
    c, s, r = Datos[t]
    # Predicciones
    # P(r,c,s|M) = P(r)P(c)P(s|r,c)
    pDatos_M0.append(      pr[r]
                         * pc[c]
                         * ps_rM0[r][s] )
    pDatos_M1.append(      pr[r]
                         * pc[c]
                         * ps_rcM1[c][r][s] )

# Revisar si los resultados son iguales
print("P(Datos|M0) =",np.prod(pDatos_M0))
print("P(Datos|M1) =",np.prod(pDatos_M1))



P(Datos|M0) = 8.234550899283273e-21
P(Datos|M1) = 3.372872048346429e-17


### **Ejercicio 1.1.4**: Expresar intuitivamente la diferencia de desempeño predictivo de los modelos

La predicción sobre un conjunto de datos necesariamente resulta ser un número muy cercano a 0. En este caso, en el que observamos tan solo 16 episodios, el modelo Base predijo el conjunto de datos con probabilidad $P(\text{Datos}|M0) = 8.23e-21$ (con 21 ceros después de la coma) y el modelo Monty Hall predijo con probabilidad $P(\text{Datos}|M1) = 3.37e-17$ (con 17 ceros después de la coma). Si seguimos agregando episodios al conjunto de datos, este número alcanza valores tan cercanos a cero que deja de ser posible representarlo en una computadora.

Existen dos formas alternativas de expresar este número, que son muy útiles además para ganar intuición respecto de la diferencia de desempeño entre modelos. 

- #### **La predicción expresada en órdenes de magnitud**

Expresar la predicción en órdenes de magnitud significa hablar en términos del exponente. Por ejemplo, el exponente de las predicciones del modelo Base es alrededor de $-21$ y el exponente del modelo Monty Hall es alrededor de $-17$, como ya mencionamos anterioramente. La función que nos devuelve el exponente de un número es el `logaritmo`. 

$$\underbrace{\log_{10} P(\text{Datos}=\{d_1, d_2, \dots, d_N \}|M)}_{\text{Exponente de la predicción conjunta}} = \log_{10} (\underbrace{P(d_1|M) P(d_2|d_1,M) \dots}_{P(\text{Datos}|M)}) \overset{*}{=}  \underbrace{\log_{10} P(d_1|M)}_{\text{Exponente de $d_1$}} + \underbrace{\log_{10} P(d_2|d_1,M) }_{\text{Exponente de $d_2$}} + \dots  $$

El exponente de la predicción conjunta se puede descomponer ($\overset{*}{=}$) como la suma de los exponentes de las prediciones individuales. Esto permite evitar los problemas de representación computacional. Además, si calculamos la diferencia de exponentes entre los modelos obtendremos la diferencia de desempeño predictivo en órdenes de magnitud, que se conoce como *log Bayes Factor*.

$$ \log_{10} \underbrace{\frac{P(\text{Datos}|M_1)}{P(\text{Datos}|M_0)}}_{\text{Bayes factor}}  = \underbrace{\log_{10} P(\text{Datos}|M_1) - \log_{10}P(\text{Datos}|M_0)}_{\text{Diferencia predicitva en ordenes de magnitud}} \approx (-17) - (-21) = 4$$

En base 10, un orden de magnitud significa 10 veces mejor, dos ordenes de magnitud significa 100 veces mejor y cuatro órdenes de magnitud significan 10000. Es decir, al modelo Monty Hall preservó 10000 veces más de creencia previa que el modelo Base. Aunque estos números parezcan extraordinarios, cuatro órdenes de magnitud se considera en el límite de una diferencia no concluytente. Cuando las bases de datos crecen, la diferencia en órdenes de magnitud contiunan creciendo, por lo que es normal ver diferencia de 10000, pero en órdenes de magnitud! En esos casos, para ganar intuición es útil calcular la predicción "típica".    

- #### **La predicción "típica" (o media geométrica)**

La media geométrica representa la predicción "típica" que hace un modelo de los datos. Decimos que es típica porque podemos reemplazar cada una de las predicciones que componen la secuencia por la media geometrica sin alterar el valor final. Es decir, 

$$P(\text{Datos}=\{d_1, d_2, \dots, d_N \}|M) = P(d_1|M) P(d_2|d_1,M) \dots =  \prod_{i\in \{1,\dots,N\}} \underbrace{(P(d_1|M) P(d_2|d_1,M) \dots )^{1/N}}_{\text{Media geométrica}} $$

Así expresada, la media geométrica tendría el mismo problema de representación computacional que señalamos al inicio de este ejercicio. Para calcularla hay que trabajar en órdenes de magnitud.

$$ 10^{\, \log_{10} (P(d_1|M) P(d_2|d_1,M) \dots )^{1/N}} = 10^{\,\frac{1}{N}\big(\log_{10} P(d_1|M) + \log_{10} P(d_2|d_1,M) + \dots \big)}$$

La expresión de la derecha puede ser calculada gracias a que es la suma de los exponentes individuales, dividido luego por $N$, la cantidad de datos. En nuestro caso tenemos $T=16$ episodios, pero la cantidad total de datos es $N=T\cdot3 = 48$, tres por cada episodio. Usaremos este número para calcular la predicción típica. 

In [84]:
# Las predicciones expresadas en órdenes de magnitud
log_evidencia_M0 = np.sum(np.log10(pDatos_M0))
log_evidencia_M1 = np.sum(np.log10(pDatos_M1))
print("Diferencia de desempeño predictivo expresado en órdenes de magnitud =", log_evidencia_M1-log_evidencia_M0)
print("La cantidad creencia que preservó el modelo Monty Hall respecto del modelo Base =", 10**(log_evidencia_M1-log_evidencia_M0))

# La predicción típica de los modelos
log_evidencia_M0 = np.sum(np.log10(pDatos_M0))
log_evidencia_M1 = np.sum(np.log10(pDatos_M1))
print("Predicción típica del modelo Base (M0) =", 10**(log_evidencia_M0/(T*3)) )
print("Predicción típica del modelo Monty Hall (M1) =", 10**(log_evidencia_M1/(T*3)) )

Diferencia de desempeño predictivo expresado en órdenes de magnitud = 3.612359947967775
La cantidad creencia que preservó el modelo Monty Hall respecto del modelo Base = 4096.000000000006
Predicción típica del modelo Base (M0) = 0.38157141418444396
Predicción típica del modelo Monty Hall (M1) = 0.45376744062979096


La predicción típica del modelo Base fue $0.382$ y la del modelo Monty Hall fue $0.454$. Dado que observamos en total $N=48$ datos ($3$ datos en cada una de los $T=16$ episodios), podemos usar la predicción típica para recuperar la predicción conjunta.

 $$P(\text{Datos}|M_0) = 0.382^{48}   \hspace{2cm}  P(\text{Datos}|M_1) = 0.454^{48}$$

En promedio (geométrico), el modelo base preserva solo el $38.2\%$ de la creencia previa con cada nueva observación, mientras que el modelo Monty Hall preserva $45.4\%$ de la creencia previa. Esta diferencia crece en cada paso temporal de forma exponencial en función de la cantidad de pasos temprales. Así es que con tan solo 48 pasos temporales el modelo Monty Hall logra preservar $4096$ veces más de creencia que el modelo Base. 

La naturaleza exponencial de la pérdida de creencia hace que diferencias mucho menores entre las predicciones típicas sean suficientes para identificar qué modelo funciona realmente mejor que otro. Por ejemplo, si la predicción típica del modelo Base hubiera sido $0.452$, preservando apenas $0.02\%$ menos de creencia previa que el modelo Monty Hall, con menos de $2000$ observaciones totales encontraríamos una diferencia de desempeño predictivo mayor entre modelos a la observada previamente en este ejercicio. Aunque la diferencia de predicción típica parezca chica, si el conjunto de datos es más grande, la diferencia de desempeño predictivo también puede ser igualmente grande. 

In [33]:
prediccionTipicaA = 0.452
prediccionTipicaB = 0.454
N = 1900
print("Diferencia de desempeño predictivo en órdenes de magnitud entre dos modelos con prediciones típicas 0.454 y 0.452 es, luego de 1900 observaciones,", np.log10(prediccionTipicaB)*N - np.log10(prediccionTipicaA)*N)

Diferencia de desempeño predictivo en órdenes de magnitud entre dos modelos con prediciones típicas 0.454 y 0.452 es, luego de 1900 observaciones, 3.6430942868714737


### **Ejercicio 1.1.5**: Calcular cómo se va actualizando la creencia de los modelos a medida que vamos agregando datos

Ahora sí. Tenemos todos los elementos necesarios para calcular cómo se va actualizando la creencia de los modelos a medida que vamos agregando datos. Esto es, queremos calcular el posterior que tenemos luego de que termina cada uno de los 16 episodios. Vamos a aprovechar que tenemos la secuencia de predicciones hechas en cada uno de los epidosios (`pDatos_M0` y `pDatos_M1`) para calcular como se va actualizando la predicción conjunta y por lo tanto la creencia de los modelos a medida que vamos agregando episodios.   

In [82]:
# Vamos a calcular la creencia conjunta como
#  
# P(Datos, Modelo) = P(Datos|Modelo) P(Modelos)
#
# el producto entre la predicción (pDatos_M0) y el prior (pM).
# Más especificamente, lo vamos a caluclar para cada paso temporal
# 
# P(Datos={(c1,s1,r1),...,(ct,st,rt)}, Modelo = m) = pDatosM[m][t] 
pDatosM = [np.cumprod(pDatos_M0) * pM[0],  # Secuencia de probabilidades conjuntas del modelo Base M0
           np.cumprod(pDatos_M1) * pM[1]]  # Secuencia de probabilidades conjuntas del modelo Monty Hall
#
print("\nP(Datos, Modelo)")
m = 0; print("  Modelo Base M0")
print("    P(Datos = { }, Modelo = M0) =", pDatosM[m][0])
print("    P(Datos = {(c1,s1,r1)}, Modelo = M0) =", pDatosM[m][1])
print("    P(Datos = {(c1,s1,r1),(c2,s2,r2)}, Modelo = M0) =", pDatosM[m][2])
m = 1; print("  Modelo Monty Hall M1")
print("    P(Datos = { }, Modelo = M1) =", pDatosM[m][0])
print("    P(Datos = {(c1,s1,r1)}, Modelo = M1) =", pDatosM[m][1])
print("    P(Datos = {(c1,s1,r1),(c2,s2,r2)}, Modelo = M1) =", pDatosM[m][2])

# Predicción de los datos hecha con la contribución de todos los modelos
# P(Datos) = P(Datos|M0) P(M0) + P(Datos|M1) P(M1) 
pDatos = pDatosM[0] + pDatosM[1]
print("\nP(Datos):")
print("    pDatos[t=1] =", pDatos[1])
print("    pDatos[t=T] =", pDatos[T])

# P(Modelo = m | Datos = {(c1,s1,r1),...,(ct,st,rt)} ) = pM_Datos[m][t]
pM_Datos = [pDatosM[0] / pDatos, # Secuencia de posterior del modelo Base M0
            pDatosM[1] / pDatos] # Secuencia de posterior del modelo Monty Hall M1
print("\nP(Modelo = m | Datos):")
print("    pM_Datos[m=0][t=T] =", pM_Datos[0][T])
print("    pM_Datos[m=1][t=T] =", pM_Datos[1][T])



P(Datos, Modelo)
  Modelo Base M0
    P(Datos = { }, Modelo = M0) = 0.5
    P(Datos = {(c1,s1,r1)}, Modelo = M0) = 0.027777777777777776
    P(Datos = {(c1,s1,r1),(c2,s2,r2)}, Modelo = M0) = 0.0015432098765432098
  Modelo Monty Hall M1
    P(Datos = { }, Modelo = M1) = 0.5
    P(Datos = {(c1,s1,r1)}, Modelo = M1) = 0.05555555555555555
    P(Datos = {(c1,s1,r1),(c2,s2,r2)}, Modelo = M1) = 0.0030864197530864196

P(Datos):
    pDatos[t=1] = 0.08333333333333333
    pDatos[t=T] = 1.6868477517181785e-17

P(Modelo = m | Datos):
    pM_Datos[m=0][t=T] = 0.000244081034903588
    pM_Datos[m=1][t=T] = 0.9997559189650964


La creencia a favor del modelo Monty Hall luego de haber visto $T=16$ episodios es $99.98\%$. Si bien no descartamos la posibilidad de que la realidad causal subyacente oculta se corresponda con el modelo Base, tenemos bastante certeza dada la información actual de que la realidad causal se parece al modelo Monty Hall. 

### **Ejercicio 1.1.6**: Graficar el valor del posterior a medida que se observan nuevos episodios

Ya tenemos guardado el valor del posterior luego de observar cada uno de los episodios. La secuencia de posterior del modelo Base la tenemos guardada en `pM_Datos[0]` y la secuencia del modelo Monty Hall la tenemos guardada en `pM_Datos[1]`.

In [ ]:
from matplotlib import pyplot as plt
plt.plot()